## Funcion normalizar texto

In [ ]:
import re
import pandas as pd

# Modificar la función de preprocesamiento para manejar NaN
def preprocesar_texto(texto):
    if pd.isna(texto):
        return ''
    texto = str(texto).lower()
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = ' '.join(texto.split())  # Eliminar espacios en blanco adicionales
    return texto

# Aplicar el preprocesamiento a la columna de descripciones en data_2021
data_2021['descripcion_normalizada'] = data_2021['descripcion_sin_saltos'].apply(preprocesar_texto)

# Verificar los primeros resultados
print(data_2021[['descripcion_sin_saltos', 'descripcion_normalizada']].head())

## Detección de duplicados por región y semestre con vectorización TD-IDF y la similaridad coseno

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Filtrar las filas que son de enero de 2021 en adelante
# data_2021 = data_2021[data_2021['publicada'] >= '2021-01-01']

# Asegurarse de que la columna 'duplicados_con' existe en el DataFrame
data_2021['duplicados_con'] = [[] for _ in range(len(data_2021))]  # Crear la columna vacía

# Definir el umbral de similaridad
umbral = 0.85

# Crear una lista para almacenar los resultados de duplicados de todas las regiones
duplicados_totales = []

# Recorrer cada región
for region, grupo_region in data_2021.groupby('region'):
    print(f"Procesando duplicados para la región: {region}")

    # Recorrer cada semestre dentro de la región
    for semestre, grupo_semestre in grupo_region.groupby('semestre'):
        print(f"Procesando duplicados para el semestre: {semestre} en la región {region}")

        # Asegurarse de que el grupo tiene al menos 2 filas para hacer comparaciones
        if len(grupo_semestre) < 2:
            continue

        # Crear el vectorizador TF-IDF para las descripciones normalizadas del grupo por semestre
        vectorizer = TfidfVectorizer()

        # Transformar las descripciones normalizadas en una matriz TF-IDF
        tfidf_matrix = vectorizer.fit_transform(grupo_semestre['descripcion_normalizada'])

        # Calcular la similaridad coseno para las descripciones del grupo
        similaridad_coseno = cosine_similarity(tfidf_matrix)

        # Crear una columna 'duplicados_con' vacía para almacenar los IDs duplicados dentro del semestre
        grupo_semestre['duplicados_con'] = [[] for _ in range(len(grupo_semestre))]

        # Encontrar duplicados dentro del semestre
        duplicados_region = []
        for i in range(similaridad_coseno.shape[0]):
            for j in range(similaridad_coseno.shape[1]):
                # Evitar comparar una fila consigo misma y evitar duplicados repetidos
                if i != j and similaridad_coseno[i, j] > umbral:
                    id_1 = grupo_semestre.iloc[i]['id']
                    id_2 = grupo_semestre.iloc[j]['id']

                    # Verificar que el duplicado aún no está en la lista de duplicados
                    if id_2 not in grupo_semestre.iloc[i]['duplicados_con']:
                        duplicados_region.append((id_1, id_2))
                        grupo_semestre.at[grupo_semestre.index[i], 'duplicados_con'].append(id_2)

                    if id_1 not in grupo_semestre.iloc[j]['duplicados_con']:
                        grupo_semestre.at[grupo_semestre.index[j], 'duplicados_con'].append(id_1)

        # Guardar los duplicados encontrados para este semestre dentro de la región
        duplicados_totales.extend(duplicados_region)

        # Actualizar el DataFrame principal con los duplicados encontrados en el grupo por semestre
        data_2021.update(grupo_semestre)

# Eliminar duplicados en las listas de duplicados
data_2021['duplicados_con'] = data_2021['duplicados_con'].apply(lambda x: list(set(x)))

# Marcar los registros duplicados en la columna 'duplicado'
data_2021['duplicado'] = data_2021['duplicados_con'].apply(lambda x: len(x) > 0)

# Mostrar los pares de duplicados encontrados en todas las regiones y semestres
print(f"Pares de duplicados encontrados en todas las regiones y semestres: {len(duplicados_totales)}")
for dup in duplicados_totales:
    print(f"Duplicado entre id {dup[0]} y id {dup[1]}")